## Partie 1 : Définition d'un point et d'un itinéraire
NB : La base de données fournie ne contient pas des noms de points et les distances entre eux , elle contient par contre une liste de points et leurs coordonées , il faut donc tirer de ces informations là la carte de points qui constituent la base de notre problème de voyageur de commerce


In [15]:
import math

class point :
  def __init__(self, x, y ):
    self.x = x
    self.y = y
  def distance(self, p ) : # fonction qui permet de calculer la distance de ce point à un autre point
    xd = self.x - p.x
    yd = self.y - p.y  # coordonnées du vecteur liant le point à l'autre
    distance = math.sqrt(xd**2 + yd**2)
    return distance
  def __repr__(self):
    return f"({self.x},{self.y})"  # la fonction rep designe une représentation "officielle" d'un objet, cela veut dire que quand je veux afficher un point il s'affichera comme un tuple de deux coordonnées


In [16]:
point(20,30) # test de la representation


(20,30)

In [ ]:
p1= point ( 20,30 )
p2 = point ( 30,40 )
p1.distance(p2)      #test de la fonction distance

14.142135623730951

# Importation du dataset

In [6]:
import pandas as pd

Remarque : Google Colab ne peut pas accéder aux chemins locaux de l'ordinateur comme /home/essoufi/..., car Colab s'exécute dans le cloud.

In [9]:
!pip install xlrd
df = pd.read_excel('2_detail_table_customers.xls')

  Obtaining dependency information for xlrd from https://files.pythonhosted.org/packages/a6/0c/c2a72d51fe56e08a08acc85d13013558a2d793028ae7385448a6ccdfae64/xlrd-2.0.1-py2.py3-none-any.whl.metadata
   ---------------------------------------- 0.0/96.5 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/96.5 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/96.5 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/96.5 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/96.5 kB ? eta -:--:--
   --------------------------------- ------ 81.9/96.5 kB 353.1 kB/s eta 0:00:01
   ---------------------------------------- 96.5/96.5 kB 345.2 kB/s eta 0:00:00


## Mapping

À partir de la classe crée et du dataset importé  , on va construire des cartes de points à parcourir dans notre problème de fil rouge , Le dataset contient 11 itinéraires donc 11 solutions au problème de voyageur de commerce chacune de ces solutions provient d'une matrice de distance différente qu'on va chercher à reconstruire  , on va chercher donc à construire 11 matrices de distances , puis construire nos propres solutions à partir de ces matrices et les comparer a celles présentes dans le dataset , cela offre une base base pour evaluer la qualité de nos solutions .  On s'interresse au deux colonnes customer_latitude et customer_longitude ,  ce qu'on va faire c'est une  Projection géographique (traduire lat/lon en coordonnées cartésiennes)

Les degrés de latitude et de longitude ne sont pas uniformes en distance (1° de longitude ≠ même distance en km selon la latitude).

Donc on doit faire une projection, et pour un mapping local (zone pas trop grande), une approximation simple fonctionne bien.

Approximation plate locale (Equirectangulaire simplifiée) :

On fixe une origine (par ex. le point le plus au sud-ouest), et on convertit en x (km Est-Ouest) et y (km Nord-Sud) :

In [10]:
import numpy as np

lat_ref = 43.41305
lon_ref = 17.87588

# fonction permettant de convertir la longitude et la latitude d'un point en coordonnées x , y
def latlon_to_xy(lat, lon, lat_ref, lon_ref):
    R = 6371  # rayon de la Terre en km
    x = (lon - lon_ref) * np.cos(np.radians((lat + lat_ref) / 2)) * (np.pi/180) * R
    y = (lat - lat_ref) * (np.pi/180) * R
    return [x, y]  # facile d'accéder aux éléments d'une liste

## test de la fonction latlon_to_xy
print(latlon_to_xy(lat_ref, lon_ref, lat_ref, lon_ref))  # ceci devrait retourner (0,0) car on a choisit les lat et lon de référence



[0.0, 0.0]


# Création des 11 matrices de distances

Pour chaque Route_Id , on va recupérer la longitude et la latitude des clients qui la compose , on va les convertir en points et les stoquer dans une liste , et à partir de cette liste , on va construire la matrice des distances de ses clients

In [11]:
df['ROUTE_ID'].value_counts()


ROUTE_ID
2970877    129
2990001    124
2922001    119
2604001    115
2958047    110
2946091    107
3016355    101
3005971     94
3027038     91
3044702     90
2939484     78
Name: count, dtype: int64

On vient d'afficher le nombre de clients desservis dans chaque route

In [12]:
route_coords = {}

for route, group in df.groupby('ROUTE_ID'):
    # Create a list of (longitude, latitude) tuples for each route
    coords = list(zip(group['CUSTOMER_LONGITUDE'], group['CUSTOMER_LATITUDE']))
    route_coords[route] = coords

print("Route_id mapped to (longitude, latitude) tuples:")
for route, coords in route_coords.items():
    print(f"Route {route}: {coords}")

Route_id mapped to (longitude, latitude) tuples:
Route 2604001: [(17.231064, 43.713521), (17.87588, 43.41305), (17.776839, 43.135963), (17.23745, 43.71295), (17.2229, 43.70716), (17.96003741, 43.08222786), (17.70152194, 43.11001558), (17.80444, 43.35266), (17.64629, 43.2072), (17.30045863, 43.48189308), (17.79902, 43.35459), (17.871369, 43.396088), (17.21969, 43.71837), (17.23185, 43.71657), (17.70165, 43.11036), (17.2317, 43.71619), (17.69928539, 43.111171), (17.80545, 43.34568), (17.82928296, 43.31353178), (17.93107, 43.08709), (17.64911, 43.18853), (17.82157, 43.36378), (17.798946, 43.343041), (17.23264032, 43.71695937), (17.699565, 43.11064), (17.71949315, 43.08876243), (17.88067, 43.26047), (17.71267, 43.11155), (17.298542261123, 43.586910367012), (17.23658, 43.71229), (17.23204, 43.64127), (17.30644, 43.63608), (17.32819, 43.4744), (17.32754731, 43.47386757), (17.813051666666, 43.34816), (17.6682, 43.20674), (17.65760923, 43.24246863), (17.69687677, 43.22780491), (17.69685531, 43

In [13]:
# Le dictionnaire va maintenant contenir les coordonnées carteziennes obtenues par mapping
route_coords_cartesian = {}
for route, coords in route_coords.items():
    cartesian_coords = [tuple(latlon_to_xy(lat, lon, lat_ref, lon_ref)) for lon, lat in coords]
    route_coords_cartesian[route] = cartesian_coords

# Affichage du résultat
print("Coordonnées cartésiennes par route_id:")
for route, coords in route_coords_cartesian.items():
    print(f"Route {route} : {coords}")

Coordonnées cartésiennes par route_id:
Route 2604001 : [(-51.954990138035356, 33.410850803817404), (0.0, 0.0), (-8.018216911486121, -30.810668639161026), (-51.44069245637541, 33.34735850070327), (-52.61556858054644, 32.70353987543155), (6.816270186427378, -36.7857435896954), (-14.118786748957163, -33.695890102675946), (-5.773371841716319, -6.715061620064699), (-18.5763934467104, -22.889475649782202), (-46.452667531554596, 7.655001230585556), (-6.21128634402526, -6.500455411640528), (-0.36442259874407323, -1.886088345744947), (-52.86930365745131, 33.950035003116874), (-51.890346314640944, 33.74988413515634), (-14.108377079029196, -33.6575923460413), (-51.90259571357252, 33.70763006303143), (-14.299757526334464, -33.567413260532696), (-5.692077070349355, -7.4912022080435685), (-3.766921029735466, -11.065921172697081), (4.469896793669338, -36.245098289059804), (-18.35104180582981, -24.965484930236137), (-4.388620757975915, -5.478574035777401), (-6.217858685865791, -7.784645619458788), (-5

In [17]:
#le dictionnaire va maintenant contenir des points pour chaque route_id
route_points = {}
for route, coords in route_coords.items():
    points = [point(*latlon_to_xy(lat, lon, lat_ref, lon_ref)) for lon, lat in coords]
    route_points[route] = points

# Affichage du résultat
print("Coordonnées cartésiennes sous forme d'objets Point par route_id:")
for route, points in route_points.items():
    print(f"Route {route}: {points}")

Coordonnées cartésiennes sous forme d'objets Point par route_id:
Route 2604001: [(-51.954990138035356,33.410850803817404), (0.0,0.0), (-8.018216911486121,-30.810668639161026), (-51.44069245637541,33.34735850070327), (-52.61556858054644,32.70353987543155), (6.816270186427378,-36.7857435896954), (-14.118786748957163,-33.695890102675946), (-5.773371841716319,-6.715061620064699), (-18.5763934467104,-22.889475649782202), (-46.452667531554596,7.655001230585556), (-6.21128634402526,-6.500455411640528), (-0.36442259874407323,-1.886088345744947), (-52.86930365745131,33.950035003116874), (-51.890346314640944,33.74988413515634), (-14.108377079029196,-33.6575923460413), (-51.90259571357252,33.70763006303143), (-14.299757526334464,-33.567413260532696), (-5.692077070349355,-7.4912022080435685), (-3.766921029735466,-11.065921172697081), (4.469896793669338,-36.245098289059804), (-18.35104180582981,-24.965484930236137), (-4.388620757975915,-5.478574035777401), (-6.217858685865791,-7.784645619458788), (

In [18]:
route_distance_matrices = {} # dictionnaire de matrices de distances pour chaque route_id
for route, points in route_points.items():
    n = len(points)
    # Initialisation d'une matrice n x n avec des zéros
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist_matrix[i, j] = points[i].distance(points[j])
    route_distance_matrices[route] = dist_matrix

# Affichage des matrices de distances
for route, matrix in route_distance_matrices.items():
    print(f"Matrice de distances pour la route {route}:")
    print(matrix)
    print()

Matrice de distances pour la route 2604001:
[[ 0.         61.77059132 77.81287555 ...  0.22717696 28.05462975
   0.28822628]
 [61.77059132  0.         31.83691418 ... 61.91114808 45.1282135
  61.88645564]
 [77.81287555 31.83691418  0.         ... 78.01168042 52.17943254
  78.02205683]
 ...
 [ 0.22717696 61.91114808 78.01168042 ...  0.         28.27863417
   0.08906235]
 [28.05462975 45.1282135  52.17943254 ... 28.27863417  0.
  28.31780564]
 [ 0.28822628 61.88645564 78.02205683 ...  0.08906235 28.31780564
   0.        ]]

Matrice de distances pour la route 2922001:
[[  0.          76.03962492  83.86393614 ... 153.17722004 152.68656347
   76.50186051]
 [ 76.03962492   0.          16.45362409 ...  86.98298743  86.59815004
   15.16541307]
 [ 83.86393614  16.45362409   0.         ...  72.67784547  72.25496055
   31.18146567]
 ...
 [153.17722004  86.98298743  72.67784547 ...   0.           0.54995318
   98.14090768]
 [152.68656347  86.59815004  72.25496055 ...   0.54995318   0.
   97.802031

Nous allons maintenant enrichir les matrices déja crées avec une colonne au début qui représente la distance entre les clients et le dépot , les coordonnées du dépot de chaque route ID se trouvent dans la table 4 de la base de données 

In [19]:
df_depots = pd.read_excel('4_detail_table_depots.xls')

In [20]:
depot_points = {} #dictionnaire qui contient les coordonnées du dépot de chaque route ID
for _, row in df_depots.iterrows():
    route_id = row['ROUTE_ID']
    lat = float(str(row['DEPOT_LATITUDE']).replace(',', '.'))
    lon = float(str(row['DEPOT_LONGITUDE']).replace(',', '.'))
    x, y = latlon_to_xy(lat, lon, lat_ref, lon_ref)
    depot_points[route_id] = point(x, y)  

# Nouveau dictionnaire pour stocker les DataFrames enrichis
route_distance_dfs = {}  #dictionnaire contenant la version finale des matrices de distances 

for route_id, matrix in route_distance_matrices.items():
    points = route_points[route_id]  # clients sous forme de points pour une route ID ( liste des clients sans le point de depot)
    depot = depot_points.get(route_id) # depot de la route ID en question  

    if depot is None:
        print(f"Aucun dépôt trouvé pour la route {route_id}")
        continue

    # Calcul des distances client-dépôt
    distances_depot = [client.distance(depot) for client in points]

    # Conversion de la matrice sans le depot en DataFrame
    df_matrix = pd.DataFrame(matrix)

    # Ajout de la colonne des distances
    df_matrix.insert(0, 'DIST_CLIENT_DEPOT', distances_depot)  # ← insérée en première position

    # Stockage
    route_distance_dfs[route_id] = df_matrix 

On a à présent finit la création des données test , il reste à exporter le dictionnaire des matrices de distances dans un fichier python à part pour les utiliser au besoin 

In [21]:
# Créer un fichier Python qui contient les matrices enrichies
with open("donnees_test.py", "w", encoding="utf-8") as f:
    f.write("route_distance_dfs = {\n")
    for route_id, df in route_distance_dfs.items():
        matrix = df.values.tolist()
        f.write(f"    {route_id}: {matrix},\n")
    f.write("}\n")
